In [3]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [4]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [5]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [6]:
def actor(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('actor', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [7]:
def generator(actions, state_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=actions, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=state_size)        
        return logits

In [8]:
def discriminator(states, actions, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=1)        
        return logits

In [26]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    actions_logits = actor(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ###############################################
    next_states_logits = generator(actions=actions_logits, hidden_size=hidden_size, state_size=state_size)
    next_states_labels = tf.nn.sigmoid(next_states)
    aloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits, 
                                                                    labels=next_states_labels))
    ####################################################
    dQs = discriminator(actions=actions_labels, hidden_size=hidden_size, states=states, action_size=action_size)
    rates = tf.reshape(rates, shape=[-1, 1])
    dloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=dQs, # GAN
                                                                   labels=rates)) # 0-1
    ####################################################
    gQs = discriminator(actions=actions_logits, hidden_size=hidden_size, states=states, action_size=action_size, 
                        reuse=True)
    dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=tf.zeros_like(gQs))) # 0-1
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs, # GAN
                                                                    labels=tf.ones_like(gQs))) # 0-1
    #####################################################
    next_actions_logits = actor(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    gQs2 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states, 
                         action_size=action_size, reuse=True)
    gQs2 = tf.reshape(gQs2, shape=[-1]) * (1-dones)
    dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs2, # GAN
                                                                    labels=tf.zeros_like(gQs2))) # 0-1
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs2, # GAN
                                                                     labels=tf.ones_like(gQs2))) # 0-1
    ##################################################### repeatable!
    next_states_logits = generator(actions=next_actions_logits, hidden_size=hidden_size, state_size=state_size, 
                                   reuse=True)
    next_actions_logits = actor(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
                                reuse=True)
    gQs3 = discriminator(actions=next_actions_logits, hidden_size=hidden_size, states=next_states_logits, 
                         action_size=action_size, reuse=True)
    dones2 = tf.concat(axis=0, values=[dones[1:], tf.ones(shape=[1])])
    gQs3 = tf.reshape(gQs3, shape=[-1]) * (1-dones2)
    dloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs3, # GAN
                                                                    labels=tf.zeros_like(gQs3))) # 0-1
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=gQs3, # GAN
                                                                     labels=tf.ones_like(gQs3))) # 0-1
    return actions_logits, aloss, dloss, aloss2, dones, dones2

In [27]:
def model_opt(a_loss, a_loss2, d_loss, a_learning_rate, d_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('actor')]
    g_vars = [var for var in t_vars if var.name.startswith('generator')]
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        d_opt = tf.train.AdamOptimizer(d_learning_rate).minimize(d_loss, var_list=d_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, d_opt, a_opt2

In [28]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, d_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.d_loss, self.a_loss2, self.dones1, self.dones2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, #model input 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.d_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        a_loss2=self.a_loss2, 
                                                        d_loss=self.d_loss,
                                                        a_learning_rate=a_learning_rate,
                                                        d_learning_rate=d_learning_rate)

In [29]:
# def sample(buffer, batch_size):
#     idx = np.random.choice(np.arange(len(buffer)), size=batch_size, replace=False)
#     return [buffer[ii] for ii in idx]

In [30]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [31]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [32]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
d_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size: 200/500 a successfull episode size
gamma = 0.99                   # future reward discount

In [33]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size,
              a_learning_rate=a_learning_rate, d_learning_rate=d_learning_rate, gamma=gamma)

# Init the memory
memory = Memory(max_size=memory_size)

In [34]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        state = env.reset()
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
# aloss_list, dloss_list, aloss2_list = [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0 # each episode
        aloss_batch, dloss_batch, aloss2_batch = [], [], []
        state = env.reset() # each episode
        num_step = 0 # each episode

        # Training steps/batches
        while True:
            # Explore (Env) or Exploit (Model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            rate = -1
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Rating the memory
            if done is True:
                rate = total_reward/500 # update rate at the end/ when episode is done
                for idx in range(num_step): # episode length
                    if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                        memory.buffer[-1-idx][-1] = rate # rate the trajectory/data
                        
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones)==1 and len(dones) >= 1 and np.max(rates) > 0:
                    # print('np.count_nonzero(dones)==1 and len(dones) >= 1 and np.max(rates) > 0: ', 
                    #       np.count_nonzero(dones), len(dones), np.max(rates))
                    break
            if np.count_nonzero(dones)!=1 and len(dones) < 1 and np.max(rates) <= 0:
                print(np.count_nonzero(dones), len(dones), np.max(rates))
                break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                feed_dict = {model.states: states, 
                                            model.actions: actions,
                                            model.next_states: next_states,
                                            model.rewards: rewards,
                                            model.dones: dones,
                                            model.rates: rates})
            dloss, _ = sess.run([model.d_loss, model.d_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _, dones1, dones2 = sess.run([model.a_loss2, model.a_opt2, model.dones1, model.dones2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print('dones:', 
            #       len(dones), np.count_nonzero(dones), 
            #       len(dones1), np.count_nonzero(dones1), 
            #       len(dones2), np.count_nonzero(dones2))
            aloss_batch.append(aloss)
            dloss_batch.append(dloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'dloss:{:.4f}'.format(np.mean(dloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        # gloss_list.append([ep, np.mean(gloss_batch)])
        # dloss_list.append([ep, np.mean(dloss_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:17.0000 R:17.0000 rate:0.0340 aloss:1.3927 dloss:2.6308 aloss2:2.1006 exploreP:0.9983
Episode:1 meanR:20.5000 R:24.0000 rate:0.0480 aloss:1.3955 dloss:2.5845 aloss2:2.1306 exploreP:0.9959
Episode:2 meanR:19.3333 R:17.0000 rate:0.0340 aloss:1.4068 dloss:2.6085 aloss2:2.1205 exploreP:0.9943
Episode:3 meanR:17.0000 R:10.0000 rate:0.0200 aloss:1.3948 dloss:2.5875 aloss2:2.1448 exploreP:0.9933
Episode:4 meanR:20.6000 R:35.0000 rate:0.0700 aloss:1.3973 dloss:2.5724 aloss2:2.1502 exploreP:0.9899
Episode:5 meanR:23.8333 R:40.0000 rate:0.0800 aloss:1.3845 dloss:2.5424 aloss2:2.1691 exploreP:0.9859
Episode:6 meanR:23.8571 R:24.0000 rate:0.0480 aloss:1.3917 dloss:2.4997 aloss2:2.1884 exploreP:0.9836
Episode:7 meanR:22.7500 R:15.0000 rate:0.0300 aloss:1.3858 dloss:2.4955 aloss2:2.2077 exploreP:0.9821
Episode:8 meanR:21.6667 R:13.0000 rate:0.0260 aloss:1.3985 dloss:2.4825 aloss2:2.2187 exploreP:0.9809
Episode:9 meanR:22.5000 R:30.0000 rate:0.0600 aloss:1.3947 dloss:2.4602 aloss2:2.2

Episode:80 meanR:21.7160 R:12.0000 rate:0.0240 aloss:1.3895 dloss:0.9088 aloss2:5.7312 exploreP:0.8403
Episode:81 meanR:21.6341 R:15.0000 rate:0.0300 aloss:1.3916 dloss:0.8333 aloss2:6.0749 exploreP:0.8391
Episode:82 meanR:21.5181 R:12.0000 rate:0.0240 aloss:1.3880 dloss:0.8062 aloss2:6.0711 exploreP:0.8381
Episode:83 meanR:21.6905 R:36.0000 rate:0.0720 aloss:1.3907 dloss:0.8186 aloss2:6.1763 exploreP:0.8351
Episode:84 meanR:21.6235 R:16.0000 rate:0.0320 aloss:1.3842 dloss:0.8048 aloss2:6.2571 exploreP:0.8338
Episode:85 meanR:21.5698 R:17.0000 rate:0.0340 aloss:1.3830 dloss:0.7881 aloss2:6.3487 exploreP:0.8324
Episode:86 meanR:21.8161 R:43.0000 rate:0.0860 aloss:1.3861 dloss:0.8206 aloss2:6.3273 exploreP:0.8289
Episode:87 meanR:21.7159 R:13.0000 rate:0.0260 aloss:1.3843 dloss:0.8059 aloss2:6.4626 exploreP:0.8278
Episode:88 meanR:21.7191 R:22.0000 rate:0.0440 aloss:1.3861 dloss:0.7862 aloss2:6.5094 exploreP:0.8260
Episode:89 meanR:21.7222 R:22.0000 rate:0.0440 aloss:1.3898 dloss:0.7448 

Episode:159 meanR:26.7700 R:36.0000 rate:0.0720 aloss:1.3729 dloss:0.6154 aloss2:10.1705 exploreP:0.6715
Episode:160 meanR:27.1900 R:70.0000 rate:0.1400 aloss:1.3782 dloss:0.6772 aloss2:10.0951 exploreP:0.6669
Episode:161 meanR:27.2000 R:23.0000 rate:0.0460 aloss:1.3752 dloss:0.6727 aloss2:9.9412 exploreP:0.6654
Episode:162 meanR:27.5200 R:46.0000 rate:0.0920 aloss:1.3711 dloss:0.5977 aloss2:10.1259 exploreP:0.6624
Episode:163 meanR:27.3200 R:25.0000 rate:0.0500 aloss:1.3661 dloss:0.6269 aloss2:10.2216 exploreP:0.6607
Episode:164 meanR:27.8900 R:78.0000 rate:0.1560 aloss:1.3703 dloss:0.6075 aloss2:10.1471 exploreP:0.6557
Episode:165 meanR:28.1100 R:35.0000 rate:0.0700 aloss:1.3660 dloss:0.6162 aloss2:10.1211 exploreP:0.6534
Episode:166 meanR:28.4800 R:63.0000 rate:0.1260 aloss:1.3698 dloss:0.6178 aloss2:10.0938 exploreP:0.6494
Episode:167 meanR:28.5700 R:21.0000 rate:0.0420 aloss:1.3783 dloss:0.6052 aloss2:10.0267 exploreP:0.6480
Episode:168 meanR:28.7600 R:32.0000 rate:0.0640 aloss:1.

Episode:238 meanR:46.6800 R:49.0000 rate:0.0980 aloss:1.3710 dloss:0.6199 aloss2:10.0315 exploreP:0.4564
Episode:239 meanR:46.8000 R:57.0000 rate:0.1140 aloss:1.3799 dloss:0.6222 aloss2:10.0066 exploreP:0.4539
Episode:240 meanR:47.0700 R:114.0000 rate:0.2280 aloss:1.3765 dloss:0.6647 aloss2:9.9258 exploreP:0.4488
Episode:241 meanR:48.0900 R:117.0000 rate:0.2340 aloss:1.3739 dloss:0.6377 aloss2:10.0179 exploreP:0.4437
Episode:242 meanR:48.7700 R:92.0000 rate:0.1840 aloss:1.3697 dloss:0.6079 aloss2:10.0632 exploreP:0.4398
Episode:243 meanR:48.7700 R:12.0000 rate:0.0240 aloss:1.3745 dloss:0.6414 aloss2:10.0229 exploreP:0.4392
Episode:244 meanR:49.6300 R:100.0000 rate:0.2000 aloss:1.3717 dloss:0.6209 aloss2:10.0367 exploreP:0.4350
Episode:245 meanR:50.2000 R:82.0000 rate:0.1640 aloss:1.3798 dloss:0.6142 aloss2:10.0451 exploreP:0.4315
Episode:246 meanR:50.2500 R:25.0000 rate:0.0500 aloss:1.3683 dloss:0.6309 aloss2:10.0548 exploreP:0.4304
Episode:247 meanR:49.6600 R:36.0000 rate:0.0720 aloss

Episode:317 meanR:86.8100 R:67.0000 rate:0.1340 aloss:1.3729 dloss:0.7548 aloss2:9.6117 exploreP:0.2284
Episode:318 meanR:87.4600 R:77.0000 rate:0.1540 aloss:1.3741 dloss:0.6591 aloss2:9.6519 exploreP:0.2267
Episode:319 meanR:87.8200 R:59.0000 rate:0.1180 aloss:1.3866 dloss:0.6657 aloss2:9.8052 exploreP:0.2254
Episode:320 meanR:88.1200 R:121.0000 rate:0.2420 aloss:1.3761 dloss:0.7035 aloss2:9.7569 exploreP:0.2229
Episode:321 meanR:87.7100 R:60.0000 rate:0.1200 aloss:1.3864 dloss:0.6439 aloss2:9.7004 exploreP:0.2216
Episode:322 meanR:87.7200 R:59.0000 rate:0.1180 aloss:1.3837 dloss:0.7009 aloss2:9.6567 exploreP:0.2203
Episode:323 meanR:88.0400 R:46.0000 rate:0.0920 aloss:1.3864 dloss:0.6388 aloss2:9.7745 exploreP:0.2194
Episode:324 meanR:87.7200 R:41.0000 rate:0.0820 aloss:1.3827 dloss:0.6676 aloss2:9.7773 exploreP:0.2185
Episode:325 meanR:87.6500 R:43.0000 rate:0.0860 aloss:1.4166 dloss:0.6609 aloss2:9.7051 exploreP:0.2176
Episode:326 meanR:87.7200 R:50.0000 rate:0.1000 aloss:1.3798 dl

...

Episode:712 meanR:44.1800 R:39.0000 rate:0.0780 aloss:1.3683 dloss:0.7286 aloss2:9.7650 exploreP:0.0451
Episode:713 meanR:44.3100 R:53.0000 rate:0.1060 aloss:1.3576 dloss:0.7056 aloss2:9.7961 exploreP:0.0450
Episode:714 meanR:44.1200 R:41.0000 rate:0.0820 aloss:1.3746 dloss:0.6837 aloss2:9.7103 exploreP:0.0448
Episode:715 meanR:43.9800 R:35.0000 rate:0.0700 aloss:1.3651 dloss:0.7423 aloss2:9.5916 exploreP:0.0447
Episode:716 meanR:43.9500 R:39.0000 rate:0.0780 aloss:1.3575 dloss:0.6429 aloss2:9.8434 exploreP:0.0446
Episode:717 meanR:44.0300 R:56.0000 rate:0.1120 aloss:1.3812 dloss:0.6770 aloss2:9.7531 exploreP:0.0444
Episode:718 meanR:44.2000 R:62.0000 rate:0.1240 aloss:1.3695 dloss:0.7167 aloss2:9.6981 exploreP:0.0442
Episode:719 meanR:44.0500 R:30.0000 rate:0.0600 aloss:1.3647 dloss:0.6309 aloss2:9.7335 exploreP:0.0441
Episode:720 meanR:43.9900 R:43.0000 rate:0.0860 aloss:1.3697 dloss:0.6919 aloss2:9.7594 exploreP:0.0439
Episode:721 meanR:43.9500 R:33.0000 rate:0.0660 aloss:1.3718 dlo

Episode:791 meanR:56.4900 R:39.0000 rate:0.0780 aloss:1.3554 dloss:0.7164 aloss2:9.6414 exploreP:0.0317
Episode:792 meanR:56.6300 R:49.0000 rate:0.0980 aloss:1.3661 dloss:0.6552 aloss2:9.5726 exploreP:0.0316
Episode:793 meanR:56.7500 R:47.0000 rate:0.0940 aloss:1.3593 dloss:0.6712 aloss2:9.7152 exploreP:0.0315
Episode:794 meanR:56.7200 R:42.0000 rate:0.0840 aloss:1.3636 dloss:0.7557 aloss2:9.6833 exploreP:0.0314
Episode:795 meanR:56.8900 R:53.0000 rate:0.1060 aloss:1.3635 dloss:0.7198 aloss2:9.5667 exploreP:0.0313
Episode:796 meanR:56.9200 R:48.0000 rate:0.0960 aloss:1.3678 dloss:0.7285 aloss2:9.6596 exploreP:0.0312
Episode:797 meanR:56.9300 R:37.0000 rate:0.0740 aloss:1.3410 dloss:0.6724 aloss2:9.7357 exploreP:0.0312
Episode:798 meanR:57.1400 R:59.0000 rate:0.1180 aloss:1.3514 dloss:0.7125 aloss2:9.6815 exploreP:0.0310
Episode:799 meanR:57.4200 R:80.0000 rate:0.1600 aloss:1.3733 dloss:0.7278 aloss2:9.5244 exploreP:0.0309
Episode:800 meanR:57.5800 R:53.0000 rate:0.1060 aloss:1.3604 dlo

Episode:870 meanR:84.0900 R:85.0000 rate:0.1700 aloss:1.3506 dloss:0.7981 aloss2:9.3336 exploreP:0.0217
Episode:871 meanR:84.8700 R:103.0000 rate:0.2060 aloss:1.3544 dloss:0.7644 aloss2:9.3060 exploreP:0.0216
Episode:872 meanR:84.3100 R:65.0000 rate:0.1300 aloss:1.3541 dloss:0.7328 aloss2:9.2922 exploreP:0.0215
Episode:873 meanR:84.7700 R:79.0000 rate:0.1580 aloss:1.3535 dloss:0.7822 aloss2:9.4043 exploreP:0.0214
Episode:874 meanR:84.4500 R:59.0000 rate:0.1180 aloss:1.3540 dloss:0.8046 aloss2:9.2692 exploreP:0.0214
Episode:875 meanR:84.1800 R:86.0000 rate:0.1720 aloss:1.3583 dloss:0.7456 aloss2:9.2996 exploreP:0.0213
Episode:876 meanR:83.8100 R:74.0000 rate:0.1480 aloss:1.3636 dloss:0.8263 aloss2:9.2218 exploreP:0.0212
Episode:877 meanR:82.7800 R:85.0000 rate:0.1700 aloss:1.3471 dloss:0.7682 aloss2:9.3141 exploreP:0.0211
Episode:878 meanR:81.8700 R:79.0000 rate:0.1580 aloss:1.3528 dloss:0.7715 aloss2:9.2774 exploreP:0.0210
Episode:879 meanR:81.6000 R:100.0000 rate:0.2000 aloss:1.3401 d

Episode:949 meanR:83.8200 R:74.0000 rate:0.1480 aloss:1.3490 dloss:0.8047 aloss2:9.1966 exploreP:0.0159
Episode:950 meanR:83.5800 R:64.0000 rate:0.1280 aloss:1.3490 dloss:0.8008 aloss2:9.0978 exploreP:0.0159
Episode:951 meanR:83.0800 R:68.0000 rate:0.1360 aloss:1.3410 dloss:0.7969 aloss2:9.1029 exploreP:0.0158
Episode:952 meanR:83.8600 R:101.0000 rate:0.2020 aloss:1.3383 dloss:0.7175 aloss2:9.2585 exploreP:0.0158
Episode:953 meanR:84.2400 R:83.0000 rate:0.1660 aloss:1.3411 dloss:0.7219 aloss2:9.3778 exploreP:0.0157
Episode:954 meanR:85.0900 R:125.0000 rate:0.2500 aloss:1.3490 dloss:0.8252 aloss2:9.2189 exploreP:0.0157
Episode:955 meanR:85.2200 R:109.0000 rate:0.2180 aloss:1.3459 dloss:0.7905 aloss2:9.2645 exploreP:0.0156
Episode:956 meanR:85.6300 R:104.0000 rate:0.2080 aloss:1.3517 dloss:0.7590 aloss2:9.2289 exploreP:0.0155
Episode:957 meanR:85.8800 R:90.0000 rate:0.1800 aloss:1.3365 dloss:0.7969 aloss2:9.2846 exploreP:0.0155
Episode:958 meanR:85.9000 R:101.0000 rate:0.2020 aloss:1.342

Episode:1028 meanR:96.9900 R:72.0000 rate:0.1440 aloss:1.3275 dloss:0.8630 aloss2:9.0160 exploreP:0.0127
Episode:1029 meanR:97.8500 R:199.0000 rate:0.3980 aloss:1.3373 dloss:0.7991 aloss2:9.0387 exploreP:0.0127
Episode:1030 meanR:99.0900 R:200.0000 rate:0.4000 aloss:1.3322 dloss:0.7935 aloss2:9.1157 exploreP:0.0126
Episode:1031 meanR:99.2300 R:89.0000 rate:0.1780 aloss:1.3294 dloss:0.8928 aloss2:9.0901 exploreP:0.0126
Episode:1032 meanR:100.0000 R:174.0000 rate:0.3480 aloss:1.3254 dloss:0.8621 aloss2:8.8864 exploreP:0.0126
Episode:1033 meanR:100.6600 R:171.0000 rate:0.3420 aloss:1.3364 dloss:0.7965 aloss2:8.9480 exploreP:0.0125
Episode:1034 meanR:100.9200 R:94.0000 rate:0.1880 aloss:1.3281 dloss:0.8825 aloss2:9.0037 exploreP:0.0125
Episode:1035 meanR:100.3900 R:56.0000 rate:0.1120 aloss:1.3375 dloss:0.8512 aloss2:8.8290 exploreP:0.0125
Episode:1036 meanR:100.7200 R:96.0000 rate:0.1920 aloss:1.3395 dloss:0.8203 aloss2:8.9647 exploreP:0.0125
Episode:1037 meanR:100.1100 R:72.0000 rate:0.1

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(gloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('G losses')`

In [ ]:
eps, arr = np.array(dloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('D losses')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 481.0
total_reward: 481.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.